In [1]:
from wdtools import *

In [2]:
# this can take about 7 minutes
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 7.0 minutes to complete


In [103]:
setID = 'Set003'
nm_to_add = 2143

In [104]:
df = combined_reindexed_data(setID=setID, nm_to_add=nm_to_add) 

In [105]:
# export = False
gdf = match_wd_data_with_taxlot(df=df, setID=setID, all_taxlot=all_taxlot, nm_to_add=nm_to_add, export=False)

In [106]:
unmatched_df = report_unmatched(gdf=gdf, setID=setID, nm_to_add=nm_to_add, mute = False)

it is about 12.38% of data in the original 735 records unmatched
there are 0 records (0.0% of the original records) without parcel id


In [118]:
unmatched_df = unmatched_df.sort_values(by=['wetdet_delin_number'], ascending=False, key=lambda x: x.map(x.value_counts()))

In [119]:
unmatched_df.to_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setID}.csv'), index=False) 

In [16]:
def get_lot_number_from_taxlot(x):
    return int(x.split('--')[1])

In [102]:
# this function only works when the coordinates are accurate and one-on-one match among WD ID, trsqq, parcel IDs, and coordinate
# limitation - one WD record (possible with multiple records with different trasqq and parcel IDs) has only one coordindate
def review_wd_record_w_coord(wd_id, county_to_check, trsqq_to_check, parcel_IDs_to_check, lon, lat, taxlot, year):
        print(f'reviewing {wd_id}')
        wd_pt  = get_point_from_lonlat(lon = lon, lat = lat)
        tID = extract_taxlot_info(wd_pt = wd_pt, taxlot = taxlot, year = year)
        trsqq, trsqq_dict, df = read_trsqq()
        trsqq_to_check_c = pad_string(trsqq_to_check)
        if "away" not in tID:
            trsqq_to_compare = trsqq_dict[tID]
            trsqq_to_compare_c = pad_string(trsqq_to_compare)
            if trsqq_to_compare_c == trsqq_to_check_c:
                print("trsqq matched, checking county code...")
                cnty_code = int(get_county_code_from_lonlat(lon, lat))
                county_to_compare = [key for key, value in cnt_dict.items() if value == cnty_code]
                # need to check the typos in the county name first
                if county_to_check == county_to_compare[0]:
                    print("county code is corrected, need to check lot numbers...")
                    lots_to_check = [int(x) for x in get_lot_numbers(parcel_IDs_to_check)]
                    lots_to_compare = df.loc[df.trsqq==trsqq_to_compare, 'ORTaxlot'].values
                    lots_to_compare = list(map(get_lot_number_from_taxlot, lots_to_compare))
                    if any([x not in lots_to_compare for x in lots_to_check]):
                        lots_to_correct = [x for x in lots_to_check if x not in lots_to_compare]
                        cor_type, cor_notes = "lot number", f'lot number {lots_to_correct} might be incorrect'
                        print("lot numbers might be wrong...")
                    else:
                        print('lot numbers seem to be correct, need some review')
                        cor_type, cor_notes = None, None
                else:
                    cor_type, cor_notes = "county", f'from {county_to_check} to {county_to_compare}'
                    print("corrected county...")
            else:
                cor_type, cor_notes = report_trsqq_correction(trsqq_to_check_c, trsqq_to_compare_c)
                print("corrected trsqq...")
        else:
            cor_type, cor_notes = "coordinate", f'coordinate might be incorrect, nearby taxlot is {tID}'
        return cor_type, cor_notes

In [101]:
def report_trsqq_correction(trsqq_to_check, trsqq_to_compare, to_correct=False):
    diff_idx, correct_trsqq_elements, errors = compare_trsqq(trsqq_to_check, trsqq_to_compare)
    if len(diff_idx) == 1:
        keylist = trsqq_correction_dict[diff_idx[0]]
        res = f'from {errors} to {correct_trsqq_elements}'
        if to_correct:
            return trsqq_to_compare
        else:
            return keylist, res
    else:
        keylist = [trsqq_correction_dict.get(key) for key in diff_idx]
        if len(keylist) == 2:
            joined_keys = ' and '.join(keylist)
            joined_errors = ' and '.join(errors)
            joined_corrections = ' and '.join(correct_trsqq_elements)
            res = f'from {joined_errors} to {joined_corrections}'
        else:
            joined_keys = join_list_elements(keylist)
            res = f'from {join_list_elements(errors)} to {join_list_elements(correct_trsqq_elements)}'
        if to_correct:
            return trsqq_to_compare
        else:
            return joined_keys, res

In [97]:
def correct_trsqq(trsqq_to_check, lon, lat, taxlot, year):
    #print(trsqq_to_check)
    wd_pt  = get_point_from_lonlat(lon = lon, lat = lat)
    tID = extract_taxlot_info(wd_pt = wd_pt, taxlot = taxlot, year = year)
    trsqq, trsqq_dict, df = read_trsqq()
    trsqq_to_check_c = pad_string(trsqq_to_check)
    trsqq_to_compare = trsqq_dict[tID]
    trsqq_to_compare_c = pad_string(trsqq_to_compare)
    res = report_trsqq_correction(trsqq_to_check_c, trsqq_to_compare_c, to_correct=True)    
    return res, tID

In [67]:
res = review_wd_record(record_ID='WD2017-0177', 
                       county_to_check='Columbia', 
                       trsqq_to_check='05N01W34CD', 
                       parcel_IDs_to_check=parcel_IDs_to_check, 
                       lon=lon, 
                       lat=lat, 
                       taxlot=all_taxlot, 
                       year=year)

reviewing record WD2017-0177


In [68]:
res

(['section number', 'QQ'],
 'section number and QQ are corrected from 34 and CD to 05 and 00 respectively')

In [37]:
def compare_trsqq(trsqq_to_check, trsqq_to_compare):
    numbers1, letters1 = separate_numbers_letters(trsqq_to_check[:-2])
    numbers2, letters2 = separate_numbers_letters(trsqq_to_compare[:-2])
    letters1.append(trsqq_to_check[-2:])
    letters2.append(trsqq_to_compare[-2:])
    trsqq_to_check_lst = remove_tuple_format(combine_lists(numbers1, letters1))
    trsqq_to_compare_lst = remove_tuple_format(combine_lists(numbers2, letters2))
    diff_idx = find_different_indices(trsqq_to_check_lst, trsqq_to_compare_lst)
    if len(diff_idx) != 0:
        if len(diff_idx) == 1:
            correct_trsqq_elements = trsqq_to_compare_lst[diff_idx[0]]
            errors = trsqq_to_check_lst[diff_idx[0]]
        else:
            correct_trsqq_elements = get_list_elements_by_index(trsqq_to_compare_lst, diff_idx)
            errors = get_list_elements_by_index(trsqq_to_check_lst, diff_idx)
    return diff_idx, correct_trsqq_elements, errors

In [52]:
trsqq_to_check='05N01W34CD'; trsqq_to_compare='05N01W0500'

In [53]:
numbers1, letters1 = separate_numbers_letters(trsqq_to_check[:-2])
numbers2, letters2 = separate_numbers_letters(trsqq_to_compare[:-2])
letters1.append(trsqq_to_check[-2:])
letters2.append(trsqq_to_compare[-2:])
trsqq_to_check_lst = remove_tuple_format(combine_lists(numbers1, letters1))
trsqq_to_compare_lst = remove_tuple_format(combine_lists(numbers2, letters2))
diff_idx = find_different_indices(trsqq_to_check_lst, trsqq_to_compare_lst)

In [29]:
setID = 'Set003'

In [108]:
unmatched = pd.read_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setID}.csv'))

In [31]:
unmatched.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [109]:
unmatched.shape

(91, 25)

In [110]:
to_review = unmatched[((unmatched.latitude.astype(str) != 'nan')|(unmatched.longitude.astype(str) != 'nan')) & (unmatched.missinglot == 'N')]

In [111]:
to_review.shape

(50, 25)

In [112]:
outdf = to_review[['wetdet_delin_number', 'trsqq', 'parcel_id', 'county', 'latitude', 'longitude', 'DecisionLink', 'record_ID', 'IDyear']]

In [113]:
outdf.loc[:,'correct_type'], outdf.loc[:,'correction'] = zip(*outdf.apply(lambda row: review_wd_record_w_coord(wd_id = row.wetdet_delin_number, 
                                                                        county_to_check = row.county, 
                                                                        trsqq_to_check = row.trsqq, 
                                                                        parcel_IDs_to_check = row.parcel_id, 
                                                                        lon = row.longitude, 
                                                                        lat = row.latitude, 
                                                                        taxlot = all_taxlot, 
                                                                        year = row.IDyear), axis = 1))

reviewing record 2155
reviewing record 2159
reviewing record 2162
reviewing record 2261
reviewing record 2262
reviewing record 2263
reviewing record 2264
reviewing record 2265
reviewing record 2266
reviewing record 2392
reviewing record 2426
reviewing record 2427
reviewing record 2428
reviewing record 2429
reviewing record 2430
reviewing record 2431
reviewing record 2432
reviewing record 2433
reviewing record 2434
reviewing record 2435
reviewing record 2436
reviewing record 2437
reviewing record 2438
reviewing record 2439
reviewing record 2440
reviewing record 2441
reviewing record 2442
reviewing record 2443
reviewing record 2444
reviewing record 2445
reviewing record 2446
reviewing record 2447
reviewing record 2448
reviewing record 2449
reviewing record 2450
reviewing record 2451
reviewing record 2452
reviewing record 2453
reviewing record 2454
reviewing record 2455
reviewing record 2456
reviewing record 2457
reviewing record 2467
reviewing record 2525
reviewing record 2537
reviewing 

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [114]:
sel = ~outdf.correct_type.isin(['county', 'lot number', 'coordinate'])

In [115]:
outdf.loc[sel, 'cor_trsqq'], outdf.loc[sel, 'ORMAP_tID'] = zip(*outdf.loc[sel,:].apply(lambda row: correct_trsqq(trsqq_to_check = row.trsqq, 
                                                                               lon = row.longitude,
                                                                               lat = row.latitude,
                                                                               taxlot = all_taxlot,
                                                                               year = row.IDyear), axis = 1))

05N01W34CD
05N02W27
07N02W17CC
08S04W23
08S04W15
08S04W23B
08S04W22
08S04W21
08S04W16
24S09E00
39S09W09
39S09W08
39S09W05D
39S09W13
39S09W14AA
39S09W15
39S09W13A
39S09W13AA
39S09W15BD
39S09W13AB
39S09W13BB
39S09W14
39S09W14AC
39S09W14BD
39S09W14AA
39S09W14AA
39S09W14AB
39S09W14BC
39S09W14BC
39S09W14BB
39S09W15AC
39S09W14BA
39S09W14AC
39S09W13BB
39S09W08D
39S09W15BB
39S09W16
39S09W08
39S09W08C
39S09W09C
39S09W16B
39S09W15BC
33S07E00
33S07E03
33S07E03
7S11W22CD
06S11W35C
18S12W02B
03S10W18


C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [100]:
outdf.to_excel(os.path.join(inpath + '\\output\\to_review\\', f'review_unmatched_{setID}.xlsx'), index=False)